<a href="https://colab.research.google.com/github/gaeun1961/DS_Basic_Analysis_Python/blob/main/01_%EB%84%A4%EC%9D%B4%EB%B2%84_api%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import sys
import urllib.request
import datetime
import time
import json
import urllib.parse

In [2]:
from google.colab import userdata
client_id = userdata.get('naver_client_id')
client_secret = userdata.get('naver_client_secret')

In [3]:
#[CODE 1]
def getRequestUrl(url):
  req= urllib.request.Request(url)
  req.add_header("X-Naver-Client-Id", client_id)
  req.add_header("X-Naver-Client-Secret", client_secret)

  try:
    response = urllib.request.urlopen(req)
    if response.getcode() == 200:
      print("[%s] Url Request Success" % datetime.datetime.now())
      return response.read().decode('utf-8')
  except Exception as e:
    print(e)
    print("[%s] Error for URL : %s" % (datetime.datetime.now(),url))
    return None

In [18]:
#[CODE 2]
def getNaverSearch(node, srcText, start, display):
  base = "https://openapi.naver.com/v1/search"
  node = "/%s.json" % node
  parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

  url = base + node + parameters
  responseDecode = getRequestUrl(url) #[CODE 1]

  if (responseDecode == None):
    return None
  else:
    return json.loads(responseDecode)

In [19]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):
  title = post['title']
  description = post['description']
  org_link = post['originallink']
  link = post['link']

  pData = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
  pDate = pData.strftime('%Y-%m-%d %H:%M:%S')

  jsonResult.append({'cnt':cnt, 'title':title, 'description':description, 'org_link':org_link, 'link':link, 'pDate':pDate})
  return

In [16]:
#[CODE 0]
def main():
  node = 'news' #크롤링 대상 노드: 네이버 '뉴스'
  srcText = input ('검색어를 입력하세요: ')
  cnt = 0
  jsonResult = []

  jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
  total = jsonResponse['total']

  while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
    for post in jsonResponse['items']:
      cnt += 1
      getPostData(post, jsonResult, cnt) #[CODE 3]

    start = jsonResponse ['start'] + jsonResponse['display']
    if start == 1001: break #네이버 뉴스는 1000개까지만 무료 제공
    jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

  print('전체 검색: %d 건'%total)

  with open('%s_naver_%s.json' % (srcText, node), 'w', encoding = 'utf8') as outfile:
    jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
    outfile.write(jsonFile)

  print("가져온 데이터 : %d 건" %(cnt))
  print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
  main()
  #소스 파일(nvCraweler..py)을 임포트하지 않고 독립적으로 실행할 경우에는 main() 즉 [CODE 0]을 호출하여 시작하라는 의미

검색어를 입력하세요: 제미나이
[2025-11-28 12:22:00.400425] Url Request Success
[2025-11-28 12:22:01.362056] Url Request Success
[2025-11-28 12:22:02.328920] Url Request Success
[2025-11-28 12:22:03.271380] Url Request Success
[2025-11-28 12:22:04.251351] Url Request Success
[2025-11-28 12:22:05.257557] Url Request Success
[2025-11-28 12:22:06.534565] Url Request Success
[2025-11-28 12:22:07.534486] Url Request Success
[2025-11-28 12:22:08.564842] Url Request Success
[2025-11-28 12:22:09.564871] Url Request Success
전체 검색: 32300 건
가져온 데이터 : 1000 건
제미나이_naver_news.json SAVED
